# Causal Inference Example: Fertilizer Effect on Crop Yield

This notebook demonstrates a basic causal inference example using synthetic data. We'll explore how controlling for confounders leads to unbiased estimates of causal effects.

**Causal Model:**
- Treatment (F): Fertilizer use (binary: 0 or 1)
- Outcome (C): Crop yield (continuous)
- Confounder (pH): Soil pH level (continuous)

We'll define a true data-generating process, then use Bayesian linear regression to estimate the causal effect of fertilizer on crop yield, both with and without controlling for the soil pH confounder.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
import warnings

# Set random seed for reproducibility
np.random.seed(42)
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("talk")

## 1. Define the True Causal Model

We'll define a data-generating process where:
1. pH affects both fertilizer use and crop yield
2. Fertilizer affects crop yield
3. The true causal effect of fertilizer on crop yield is known

This creates a classic confounding scenario where pH is a common cause of both treatment and outcome.

In [ ]:
# Define parameters for our data generating process
n_samples = 1000  # Number of observations
true_effect_fertilizer = 5  # True causal effect of fertilizer on crop yield

# Generate soil pH levels (the confounder)
pH = np.random.normal(loc=7, scale=1, size=n_samples)

# Generate fertilizer use (treatment) influenced by pH
# Higher pH makes fertilizer use more likely
prob_fertilizer = 1 / (1 + np.exp(-(pH - 7)))  # Logistic function
fertilizer = np.random.binomial(n=1, p=prob_fertilizer, size=n_samples)

# Generate crop yield (outcome) influenced by both pH and fertilizer
# pH has a nonlinear effect on yield, with optimal growth around pH 6.5
pH_effect = -2 * (pH - 6.5)**2 + 5
noise = np.random.normal(0, 2, n_samples)
crop_yield = 10 + pH_effect + true_effect_fertilizer * fertilizer + noise

# Create a DataFrame with all variables
data = pd.DataFrame({
    'pH': pH,
    'fertilizer': fertilizer,
    'crop_yield': crop_yield
})

# Display the first few rows
data.head()

## 2. Explore the Data

Let's visualize the relationships between our variables to understand the confounding structure.

In [ ]:
# Plot distributions and relationships
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot pH distribution
sns.histplot(data=data, x='pH', kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Distribution of Soil pH')

# Plot relationship between pH and fertilizer
sns.boxplot(data=data, x='fertilizer', y='pH', ax=axes[0, 1])
axes[0, 1].set_title('Soil pH by Fertilizer Use')

# Plot relationship between fertilizer and crop yield
sns.boxplot(data=data, x='fertilizer', y='crop_yield', ax=axes[1, 0])
axes[1, 0].set_title('Crop Yield by Fertilizer Use')

# Plot relationship between pH and crop yield, colored by fertilizer
sns.scatterplot(data=data, x='pH', y='crop_yield', hue='fertilizer', ax=axes[1, 1])
axes[1, 1].set_title('Crop Yield vs pH by Fertilizer Use')

plt.tight_layout()
plt.show()

# Calculate some summary statistics to see the confounding effect
print("Average crop yield without fertilizer:", data[data.fertilizer == 0].crop_yield.mean())
print("Average crop yield with fertilizer:", data[data.fertilizer == 1].crop_yield.mean())
print("Observed difference (naive estimate):", 
      data[data.fertilizer == 1].crop_yield.mean() - data[data.fertilizer == 0].crop_yield.mean())
print("True causal effect:", true_effect_fertilizer)

## 3. Bayesian Linear Regression Models

Now we'll implement two Bayesian linear regression models:

1. **Naive model**: Ignores the confounder (pH)
2. **Adjusted model**: Controls for the confounder (pH)

We'll compare how well each model recovers the true causal effect of fertilizer on crop yield.

In [ ]:
# Standardize variables for better MCMC convergence
data_std = data.copy()
data_std['pH_std'] = (data['pH'] - data['pH'].mean()) / data['pH'].std()
data_std['crop_yield_std'] = (data['crop_yield'] - data['crop_yield'].mean()) / data['crop_yield'].std()

# 1. Naive model (ignoring the confounder)
with pm.Model() as naive_model:
    # Priors
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta_fertilizer = pm.Normal('beta_fertilizer', mu=0, sigma=10)
    sigma = pm.HalfNormal('sigma', sigma=5)
    
    # Linear model
    mu = alpha + beta_fertilizer * data_std['fertilizer']
    
    # Likelihood
    y = pm.Normal('y', mu=mu, sigma=sigma, observed=data_std['crop_yield_std'])
    
    # Inference
    naive_trace = pm.sample(2000, tune=1000, return_inferencedata=True)

# 2. Adjusted model (controlling for the confounder)
with pm.Model() as adjusted_model:
    # Priors
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta_fertilizer = pm.Normal('beta_fertilizer', mu=0, sigma=10)
    beta_pH = pm.Normal('beta_pH', mu=0, sigma=10)
    beta_pH_squared = pm.Normal('beta_pH_squared', mu=0, sigma=10)  # Allow for nonlinear pH effect
    sigma = pm.HalfNormal('sigma', sigma=5)
    
    # Linear model with pH and pH squared to capture nonlinear effects
    mu = (alpha + 
          beta_fertilizer * data_std['fertilizer'] + 
          beta_pH * data_std['pH_std'] + 
          beta_pH_squared * data_std['pH_std']**2)
    
    # Likelihood
    y = pm.Normal('y', mu=mu, sigma=sigma, observed=data_std['crop_yield_std'])
    
    # Inference
    adjusted_trace = pm.sample(2000, tune=1000, return_inferencedata=True)

## 4. Compare Model Results

Now we'll examine the posterior distributions of the fertilizer effect from both models and compare them to the true causal effect.

In [ ]:
# Extract the posterior samples for the fertilizer effect
naive_effect = naive_trace.posterior['beta_fertilizer'].values.flatten()
adjusted_effect = adjusted_trace.posterior['beta_fertilizer'].values.flatten()

# Rescale effects to original units (undo standardization)
std_y = data['crop_yield'].std()
naive_effect_rescaled = naive_effect * std_y
adjusted_effect_rescaled = adjusted_effect * std_y

# Plot the posterior distributions
plt.figure(figsize=(12, 6))
sns.kdeplot(naive_effect_rescaled, fill=True, label='Naive Model (Ignoring pH)', alpha=0.7)
sns.kdeplot(adjusted_effect_rescaled, fill=True, label='Adjusted Model (Controlling for pH)', alpha=0.7)
plt.axvline(x=true_effect_fertilizer, color='red', linestyle='--', label='True Causal Effect')
plt.title('Posterior Distributions of Fertilizer Effect')
plt.xlabel('Effect on Crop Yield')
plt.ylabel('Density')
plt.legend()
plt.show()

# Calculate summary statistics
print("Naive model effect estimate: {:.2f} (95% CI: [{:.2f}, {:.2f}])".format(
    naive_effect_rescaled.mean(),
    np.percentile(naive_effect_rescaled, 2.5),
    np.percentile(naive_effect_rescaled, 97.5)
))

print("Adjusted model effect estimate: {:.2f} (95% CI: [{:.2f}, {:.2f}])".format(
    adjusted_effect_rescaled.mean(),
    np.percentile(adjusted_effect_rescaled, 2.5),
    np.percentile(adjusted_effect_rescaled, 97.5)
))

print("True causal effect: {:.2f}".format(true_effect_fertilizer))

## 5. Visualize the Posterior Traces

Let's examine the MCMC traces to ensure our models converged properly.

In [ ]:
# Plot traces for naive model
az.plot_trace(naive_trace, var_names=['beta_fertilizer'])
plt.suptitle('Naive Model Trace (Ignoring pH)')
plt.tight_layout()
plt.show()

# Plot traces for adjusted model
az.plot_trace(adjusted_trace, var_names=['beta_fertilizer'])
plt.suptitle('Adjusted Model Trace (Controlling for pH)')
plt.tight_layout()
plt.show()

## 6. Explore the Full Posterior Results

Let's look at all parameters in our models to better understand the relationships.

In [ ]:
# Forest plot for naive model
az.plot_forest(naive_trace, var_names=['alpha', 'beta_fertilizer', 'sigma'], 
               figsize=(10, 4), combined=True)
plt.title('Naive Model Parameters')
plt.show()

# Forest plot for adjusted model
az.plot_forest(adjusted_trace, var_names=['alpha', 'beta_fertilizer', 'beta_pH', 'beta_pH_squared', 'sigma'], 
               figsize=(10, 6), combined=True)
plt.title('Adjusted Model Parameters')
plt.show()

# Summary statistics
naive_summary = az.summary(naive_trace, var_names=['alpha', 'beta_fertilizer', 'sigma'])
adjusted_summary = az.summary(adjusted_trace, var_names=['alpha', 'beta_fertilizer', 'beta_pH', 'beta_pH_squared', 'sigma'])

print("Naive Model Summary:")
print(naive_summary)
print("\nAdjusted Model Summary:")
print(adjusted_summary)

## 7. Conclusions

From our analysis, we can draw the following conclusions:

1. **Naive model (ignoring pH)**: When we ignore the confounder (soil pH), our estimate of the fertilizer effect is biased. The posterior distribution of the effect does not correctly center on the true causal effect.

2. **Adjusted model (controlling for pH)**: When we control for the confounder, our estimate is much closer to the true causal effect, and the true value falls within our credible interval.

This example demonstrates a key principle in causal inference: **controlling for confounders is essential for obtaining unbiased estimates of causal effects**. When we ignore confounders, we get a misleading picture of the relationship between treatment and outcome.

In this specific case, pH affects both the likelihood of using fertilizer and the crop yield directly. Without accounting for pH, we cannot separate the direct effect of fertilizer from the indirect relationship through pH.